In [1]:
from keras.models import Model, Input, save_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras import losses
import h5py

DATA_FILE = 'data.h5'

Using TensorFlow backend.


In [2]:
with h5py.File(DATA_FILE, 'r') as f:
    
    images_training = list(f['images_training'])
    steering_training = list(f['steering_training'])
    labels_training = list(f['label_training'])
        
    images_testing = list(f['images_testing'])
    steering_testing = list(f['steering_testing'])
    labels_testing = list(f['label_testing'])

In [3]:
activation = 'relu'

# TODO: remodel network using steering data as additional input

input_shape = (64, 64, 3)
pic_input = Input(shape=input_shape)

img_stack = Conv2D(64, (3, 3), name='convolution0', padding='same', activation=activation)(pic_input)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(64, (3, 3), name='convolution1', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(32, (3, 3), name='convolution2', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(32, (3, 3), name='convolution3', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(16, (3, 3), name='convolution4', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Flatten()(img_stack)
img_stack = Dropout(0.25)(img_stack)

dense_stack = Dense(128, activation=activation, name='dense0')(img_stack)
dense_stack = Dropout(0.25)(dense_stack)
dense_stack = Dense(32, activation=activation, name='dense1')(dense_stack)
dense_stack = Dense(1, name='output', activation='tanh')(dense_stack)

adam = Adam()
model = Model(inputs=[pic_input], outputs=dense_stack)
model.compile(optimizer=adam, loss='mse')


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
convolution0 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
convolution1 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
convolution2 (Conv2D)        (None, 16, 16, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 32)          0         
__________

In [5]:
from generator import DriveDataGenerator

plateau_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
model_checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)
tensorboard_callback = TensorBoard(log_dir='./logs')

resize_dims = (input_shape[0], input_shape[1])
train_generator = DriveDataGenerator(images_training, labels_training, resize_dims=resize_dims)
validation_generator = DriveDataGenerator(images_testing, labels_testing, resize_dims=resize_dims)

model.fit_generator(generator=train_generator, validation_data=validation_generator,
                    epochs=500, verbose=1, 
                    callbacks=[model_checkpoint, tensorboard_callback])

Epoch 1/500
244/244 [==============================] - 8s 34ms/step - loss: 0.2918 - val_loss: 0.1617
Epoch 2/500
244/244 [==============================] - 6s 23ms/step - loss: 0.1657 - val_loss: 0.1452
Epoch 3/500
244/244 [==============================] - 6s 23ms/step - loss: 0.1543 - val_loss: 0.1372
Epoch 4/500
244/244 [==============================] - 6s 23ms/step - loss: 0.1438 - val_loss: 0.1326
Epoch 5/500
244/244 [==============================] - 6s 24ms/step - loss: 0.1382 - val_loss: 0.1226
Epoch 6/500
244/244 [==============================] - 6s 25ms/step - loss: 0.1401 - val_loss: 0.1265
Epoch 7/500
244/244 [==============================] - 6s 24ms/step - loss: 0.1354 - val_loss: 0.1218
Epoch 8/500
244/244 [==============================] - 6s 25ms/step - loss: 0.1329 - val_loss: 0.1257
Epoch 9/500
244/244 [==============================] - 6s 25ms/step - loss: 0.1294 - val_loss: 0.1281
Epoch 10/500
244/244 [==============================] - 7s 27ms/step - loss: 0.132

244/244 [==============================] - 7s 30ms/step - loss: 0.1059 - val_loss: 0.1284
Epoch 81/500
244/244 [==============================] - 8s 31ms/step - loss: 0.1175 - val_loss: 0.1066
Epoch 82/500
244/244 [==============================] - 7s 28ms/step - loss: 0.1044 - val_loss: 0.1080
Epoch 83/500
244/244 [==============================] - 7s 28ms/step - loss: 0.1043 - val_loss: 0.1060
Epoch 84/500
244/244 [==============================] - 7s 29ms/step - loss: 0.1019 - val_loss: 0.1085
Epoch 85/500
244/244 [==============================] - 7s 28ms/step - loss: 0.1071 - val_loss: 0.1023
Epoch 86/500
244/244 [==============================] - 6s 26ms/step - loss: 0.1036 - val_loss: 0.1016
Epoch 87/500
244/244 [==============================] - 6s 26ms/step - loss: 0.1013 - val_loss: 0.1035
Epoch 88/500
244/244 [==============================] - 7s 27ms/step - loss: 0.1014 - val_loss: 0.1043
Epoch 89/500
244/244 [==============================] - 6s 25ms/step - loss: 0.1013 - 

244/244 [==============================] - 8s 32ms/step - loss: 0.0998 - val_loss: 0.0952
Epoch 160/500
244/244 [==============================] - 7s 28ms/step - loss: 0.0982 - val_loss: 0.1151
Epoch 161/500
244/244 [==============================] - 8s 33ms/step - loss: 0.0965 - val_loss: 0.0926
Epoch 162/500
244/244 [==============================] - 7s 30ms/step - loss: 0.0947 - val_loss: 0.0935
Epoch 163/500
244/244 [==============================] - 7s 31ms/step - loss: 0.0941 - val_loss: 0.0969
Epoch 164/500
244/244 [==============================] - 8s 31ms/step - loss: 0.0945 - val_loss: 0.0995
Epoch 165/500
244/244 [==============================] - 7s 29ms/step - loss: 0.0941 - val_loss: 0.1008
Epoch 166/500
244/244 [==============================] - 8s 31ms/step - loss: 0.0939 - val_loss: 0.0921
Epoch 167/500
244/244 [==============================] - 8s 32ms/step - loss: 0.0974 - val_loss: 0.1128
Epoch 168/500
244/244 [==============================] - 8s 33ms/step - loss: 

Epoch 238/500
244/244 [==============================] - 6s 24ms/step - loss: 0.1024 - val_loss: 0.0990
Epoch 239/500
244/244 [==============================] - 6s 25ms/step - loss: 0.0949 - val_loss: 0.1009
Epoch 240/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0916 - val_loss: 0.0936
Epoch 241/500
244/244 [==============================] - 6s 25ms/step - loss: 0.0879 - val_loss: 0.0949
Epoch 242/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0883 - val_loss: 0.0954
Epoch 243/500
244/244 [==============================] - 6s 25ms/step - loss: 0.0911 - val_loss: 0.0956
Epoch 244/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0941 - val_loss: 0.0967
Epoch 245/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0901 - val_loss: 0.0939
Epoch 246/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0935 - val_loss: 0.0931
Epoch 247/500
244/244 [==============================] - 6s 25ms

Epoch 317/500
244/244 [==============================] - 8s 31ms/step - loss: 0.0844 - val_loss: 0.0955
Epoch 318/500
244/244 [==============================] - 8s 32ms/step - loss: 0.0867 - val_loss: 0.0914
Epoch 319/500
244/244 [==============================] - 7s 30ms/step - loss: 0.0872 - val_loss: 0.0968
Epoch 320/500
244/244 [==============================] - 7s 31ms/step - loss: 0.0850 - val_loss: 0.1009
Epoch 321/500
244/244 [==============================] - 6s 25ms/step - loss: 0.0848 - val_loss: 0.0950
Epoch 322/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0857 - val_loss: 0.1003
Epoch 323/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0857 - val_loss: 0.0931
Epoch 324/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0886 - val_loss: 0.0972
Epoch 325/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0843 - val_loss: 0.0943
Epoch 326/500
244/244 [==============================] - 6s 24ms

Epoch 396/500
244/244 [==============================] - 6s 25ms/step - loss: 0.0843 - val_loss: 0.0907
Epoch 397/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0811 - val_loss: 0.1053
Epoch 398/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0828 - val_loss: 0.0877
Epoch 399/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0820 - val_loss: 0.0946
Epoch 400/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0883 - val_loss: 0.0914
Epoch 401/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0801 - val_loss: 0.0895
Epoch 402/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0837 - val_loss: 0.0949
Epoch 403/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0798 - val_loss: 0.0962
Epoch 404/500
244/244 [==============================] - 6s 24ms/step - loss: 0.0804 - val_loss: 0.0926
Epoch 405/500
244/244 [==============================] - 6s 25ms

Epoch 475/500
244/244 [==============================] - 6s 26ms/step - loss: 0.0788 - val_loss: 0.1034
Epoch 476/500
244/244 [==============================] - 7s 27ms/step - loss: 0.0832 - val_loss: 0.0991
Epoch 477/500
244/244 [==============================] - 6s 25ms/step - loss: 0.0814 - val_loss: 0.0939
Epoch 478/500
244/244 [==============================] - 6s 26ms/step - loss: 0.0790 - val_loss: 0.0942
Epoch 479/500
244/244 [==============================] - 7s 28ms/step - loss: 0.0801 - val_loss: 0.0976
Epoch 480/500
244/244 [==============================] - 8s 33ms/step - loss: 0.0833 - val_loss: 0.0942
Epoch 481/500
244/244 [==============================] - 7s 29ms/step - loss: 0.0828 - val_loss: 0.0928
Epoch 482/500
244/244 [==============================] - 6s 26ms/step - loss: 0.0777 - val_loss: 0.0910
Epoch 483/500
244/244 [==============================] - 7s 31ms/step - loss: 0.0826 - val_loss: 0.0927
Epoch 484/500
244/244 [==============================] - 7s 29ms

In [6]:
model.save('model.h5')